# GloVE

Let's work on implementation of GloVE.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

## 1. Load data

In [6]:
import ssl
import nltk

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('brown')


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [7]:
from nltk.corpus import brown

brown.categories()
corpus = brown.sents(categories="news")
corpus

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]

In [8]:
#get word sequences and unique words
flatten = lambda l: [item for sublist in l for item in sublist]
vocab = list(set(flatten(corpus)))
vocab

['prosecutors',
 'lightness',
 'comic',
 "Officers'",
 'collapsed',
 'Westphalia',
 'understand',
 'softened',
 'Kathleen',
 'schoolers',
 'plaintiffs',
 'valve',
 'common',
 'Emmert',
 'cope',
 'flows',
 'Rylie',
 'pad',
 'Crumlish',
 'fairways',
 'Metal',
 'Worth',
 'Hempstead',
 'At',
 'Nothing',
 'kicks',
 'Can',
 'sometimes',
 'veteran',
 'mentioned',
 'assessments',
 'Cover',
 'fifty',
 'assertion',
 'administrators',
 'Malcolm',
 'Mich.',
 'boatload',
 'Pakistan',
 'masterful',
 '4,500',
 'grassroots',
 '30,000',
 'smelts',
 'gather',
 'ratification',
 'undeveloped',
 'enmity',
 'Democratic',
 'Sept.',
 '1963',
 'Frank',
 'enforcement',
 'riot',
 'afternoon',
 'necessitate',
 'Lane',
 'whether',
 'Blume',
 'placed',
 'conservatives',
 'Bob',
 'entertainment',
 'motel-keepers',
 'Lynn',
 'barber',
 'youthful',
 'stolen',
 'enrollment',
 'ogled',
 '37,679',
 'title',
 'accelerated',
 'Tex.',
 'agree',
 'hostess',
 'fundamentals',
 'approximately',
 'staggered',
 'Ed',
 'Alex',
 'i

In [9]:
len(vocab)

14394

In [10]:
vocab.append('<UNK>')


In [11]:
#create handy mapping between integer and word
word2index = {w: i for i, w in enumerate(vocab)}
word2index['Country']

3540

In [12]:
len(vocab)


14395

In [13]:
index2word = {v:k for k, v in word2index.items()}
index2word[14394]

'<UNK>'

## 2. Build Co-occurence Matrix X

Here, we need to count the co-occurence of two words given some window size.  We gonna use window size of 1.

In [14]:
from collections import Counter

X_i = Counter(flatten(corpus))
X_i

Counter({'The': 806,
         'Fulton': 14,
         'County': 35,
         'Grand': 6,
         'Jury': 2,
         'said': 402,
         'Friday': 41,
         'an': 300,
         'investigation': 9,
         'of': 2849,
         "Atlanta's": 4,
         'recent': 20,
         'primary': 17,
         'election': 38,
         'produced': 6,
         '``': 732,
         'no': 109,
         'evidence': 17,
         "''": 702,
         'that': 802,
         'any': 90,
         'irregularities': 3,
         'took': 47,
         'place': 25,
         '.': 4030,
         'jury': 44,
         'further': 16,
         'in': 1893,
         'term-end': 1,
         'presentments': 1,
         'the': 5580,
         'City': 44,
         'Executive': 6,
         'Committee': 37,
         ',': 5188,
         'which': 244,
         'had': 279,
         'over-all': 2,
         'charge': 17,
         'deserves': 3,
         'praise': 2,
         'and': 2146,
         'thanks': 6,
         'Atlanta': 14,

In [15]:
window_size = 2
skip_grams = []

for doc in corpus:
    for i in range(window_size, len(doc)-window_size):
        center = doc[i]
        outside = [doc[i-1], doc[i+1]]
        for each_out in outside:
            skip_grams.append((center, each_out))
            #center, outside1;   center, outside2

skip_grams

[('County', 'Fulton'),
 ('County', 'Grand'),
 ('Grand', 'County'),
 ('Grand', 'Jury'),
 ('Jury', 'Grand'),
 ('Jury', 'said'),
 ('said', 'Jury'),
 ('said', 'Friday'),
 ('Friday', 'said'),
 ('Friday', 'an'),
 ('an', 'Friday'),
 ('an', 'investigation'),
 ('investigation', 'an'),
 ('investigation', 'of'),
 ('of', 'investigation'),
 ('of', "Atlanta's"),
 ("Atlanta's", 'of'),
 ("Atlanta's", 'recent'),
 ('recent', "Atlanta's"),
 ('recent', 'primary'),
 ('primary', 'recent'),
 ('primary', 'election'),
 ('election', 'primary'),
 ('election', 'produced'),
 ('produced', 'election'),
 ('produced', '``'),
 ('``', 'produced'),
 ('``', 'no'),
 ('no', '``'),
 ('no', 'evidence'),
 ('evidence', 'no'),
 ('evidence', "''"),
 ("''", 'evidence'),
 ("''", 'that'),
 ('that', "''"),
 ('that', 'any'),
 ('any', 'that'),
 ('any', 'irregularities'),
 ('irregularities', 'any'),
 ('irregularities', 'took'),
 ('took', 'irregularities'),
 ('took', 'place'),
 ('further', 'jury'),
 ('further', 'said'),
 ('said', 'furthe

In [16]:
X_ik_skipgrams = Counter(skip_grams)
X_ik_skipgrams

Counter({('County', 'Fulton'): 6,
         ('County', 'Grand'): 1,
         ('Grand', 'County'): 1,
         ('Grand', 'Jury'): 1,
         ('Jury', 'Grand'): 2,
         ('Jury', 'said'): 1,
         ('said', 'Jury'): 1,
         ('said', 'Friday'): 4,
         ('Friday', 'said'): 3,
         ('Friday', 'an'): 1,
         ('an', 'Friday'): 1,
         ('an', 'investigation'): 3,
         ('investigation', 'an'): 4,
         ('investigation', 'of'): 4,
         ('of', 'investigation'): 5,
         ('of', "Atlanta's"): 1,
         ("Atlanta's", 'of'): 1,
         ("Atlanta's", 'recent'): 1,
         ('recent', "Atlanta's"): 1,
         ('recent', 'primary'): 1,
         ('primary', 'recent'): 1,
         ('primary', 'election'): 2,
         ('election', 'primary'): 2,
         ('election', 'produced'): 1,
         ('produced', 'election'): 1,
         ('produced', '``'): 1,
         ('``', 'produced'): 1,
         ('``', 'no'): 1,
         ('no', '``'): 1,
         ('no', 'evidence'): 2

### Weighting function

GloVe includes a weighting function to scale down too frequent words.

<img src = "../figures/glove_weighting_func.png" width=400>

In [17]:
def weighting(w_i, w_j, X_ik):

    #check whether the co-occurences between w_i and w_j is available
    try:
        x_ij = X_ik[(w_i, w_j)]
        #if not exist, then set to 1 "laplace smoothing"
    except:
        x_ij = 1

    #set xmax
    x_max = 100
    #set alpha
    alpha = 0.75

    #if co-ocurrence does not exceeed xmax, then just multiply with some alpha
    if x_ij < x_max:
        result = (x_ij / x_max)**alpha
    #otherwise, set to 1
    else:
        result = 1

    return result

In [18]:
from itertools import combinations_with_replacement

X_ik = {} #keeping the co-occurences
weighting_dic = {} #already scale the co-occurences using the weighting function

for bigram in combinations_with_replacement(vocab, 2):
    if X_ik_skipgrams.get(bigram):  #if the pair exists in our corpus
        co = X_ik_skipgrams[bigram]
        X_ik[bigram] = co + 1 #for stability
        X_ik[(bigram[1], bigram[0])] = co + 1 #basically apple, banana = banana, apple
    else:
        pass

    weighting_dic[bigram] = weighting(bigram[0], bigram[1], X_ik)
    weighting_dic[(bigram[1], bigram[0])] = weighting(bigram[1], bigram[0], X_ik)

## 3. Prepare train data

In [19]:
import math

def random_batch(batch_size, word_sequence, skip_grams, X_ik, weighting_dic):

    random_inputs, random_labels, random_coocs, random_weightings = [], [], [], []

    #convert our skipgrams to id
    skip_grams_id = [(word2index[skip_gram[0]], word2index[skip_gram[1]]) for skip_gram in skip_grams]

    #randomly choose indexes based on batch size
    random_index = np.random.choice(range(len(skip_grams_id)), batch_size, replace=False)

    #get the random input and labels
    for index in random_index:
        random_inputs.append([skip_grams_id[index][0]])
        random_labels.append([skip_grams_id[index][1]])
        #coocs
        pair = skip_grams[index] #e.g., ('banana', 'fruit')
        try:
            cooc = X_ik[pair]
        except:
            cooc = 1
        random_coocs.append([math.log(cooc)])

        #weightings
        weighting = weighting_dic[pair]
        random_weightings.append([weighting])

    return np.array(random_inputs), np.array(random_labels), np.array(random_coocs), np.array(random_weightings)

### Testing the method

In [20]:
batch_size = 2
x, y, cooc, weighting = random_batch(batch_size, corpus, skip_grams, X_ik, weighting_dic)

In [21]:
x

array([[6911],
       [ 112]])

In [22]:
y

array([[13628],
       [ 7652]])

In [23]:
cooc

array([[1.09861229],
       [1.09861229]])

In [24]:
weighting

array([[0.07208434],
       [0.07208434]])

## 4. Model

<img src ="../figures/glove.png" width=400>

In [25]:
class Glove(nn.Module):

    def __init__(self, voc_size, emb_size, word2index):
        super(Glove, self).__init__()
        self.center_embedding  = nn.Embedding(voc_size, emb_size)
        self.outside_embedding = nn.Embedding(voc_size, emb_size)

        self.center_bias       = nn.Embedding(voc_size, 1)
        self.outside_bias      = nn.Embedding(voc_size, 1)

        self.word2index = word2index


    def forward(self, center, outside, coocs, weighting):
        center_embeds  = self.center_embedding(center) #(batch_size, 1, emb_size)
        outside_embeds = self.outside_embedding(outside) #(batch_size, 1, emb_size)

        center_bias    = self.center_bias(center).squeeze(1)
        target_bias    = self.outside_bias(outside).squeeze(1)

        inner_product  = outside_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2)
        #(batch_size, 1, emb_size) @ (batch_size, emb_size, 1) = (batch_size, 1, 1) = (batch_size, 1)

        loss = weighting * torch.pow(inner_product + center_bias + target_bias - coocs, 2)

        return torch.sum(loss)

    #let's write a function to get embedding given a word
    def get_embed(self, word):
        word2index = self.word2index

        try:
            index = word2index[word]
        except:
            index = word2index['<UNK>']

        word = torch.LongTensor([index])

        embed_c = self.center_embedding(word)
        embed_o = self.outside_embedding(word)
        embed   = (embed_c + embed_o) / 2

        return embed[0][0].item(), embed[0][1].item()

In [26]:
#test our system
voc_size = len(vocab)
emb_size = 2
model = Glove(voc_size, emb_size, word2index)

In [27]:
x_tensor = torch.LongTensor(x)
y_tensor = torch.LongTensor(y)
cooc_tensor = torch.FloatTensor(cooc)
weighting_tensor = torch.FloatTensor(weighting)

In [28]:
loss = model(x_tensor, y_tensor, cooc_tensor, weighting_tensor)

In [29]:
loss

tensor(0.2232, grad_fn=<SumBackward0>)

## 5. Training

In [30]:
batch_size     = 10 # mini-batch size
embedding_size = 2 #so we can later plot
model          = Glove(voc_size, embedding_size, word2index)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [31]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [32]:
import time

# Training
num_epochs = 100
time_start= time.time()

for epoch in range(num_epochs):
    start = time.time()

    input_batch, target_batch, cooc_batch, weighting_batch = random_batch(batch_size, corpus, skip_grams, X_ik, weighting_dic)
    input_batch  = torch.LongTensor(input_batch)         #[batch_size, 1]
    target_batch = torch.LongTensor(target_batch)        #[batch_size, 1]
    cooc_batch   = torch.FloatTensor(cooc_batch)         #[batch_size, 1]
    weighting_batch = torch.FloatTensor(weighting_batch) #[batch_size, 1]

    optimizer.zero_grad()
    loss = model(input_batch, target_batch, cooc_batch, weighting_batch)

    loss.backward()
    optimizer.step()
    end = time.time()
    epoch_mins, epoch_secs = epoch_time(start, end)
    if (epoch + 1) % 10 == 0:
        print(f"Epoch: {epoch + 1} | cost: {loss:.6f} | time: {epoch_mins}m {epoch_secs}s")

# Measure total training time and print results
training_time_total = time.time() - time_start
# Calculate minutes and seconds for readability
minutes, seconds = divmod(int(training_time_total), 60)

print(f"\nComplete: \nTotal Loss: {loss:2.2f} | Time Taken: {minutes} minutes and {seconds} seconds")

Epoch: 10 | cost: 6.710097 | time: 0m 0s
Epoch: 20 | cost: 47.699940 | time: 0m 0s
Epoch: 30 | cost: 25.986183 | time: 0m 0s
Epoch: 40 | cost: 6.472117 | time: 0m 0s
Epoch: 50 | cost: 49.505863 | time: 0m 0s
Epoch: 60 | cost: 15.390682 | time: 0m 0s
Epoch: 70 | cost: 13.684435 | time: 0m 0s
Epoch: 80 | cost: 16.546980 | time: 0m 0s
Epoch: 90 | cost: 130.537857 | time: 0m 0s
Epoch: 100 | cost: 3.791770 | time: 0m 0s

Complete: 
Total Loss: 3.79 | Time Taken: 0 minutes and 5 seconds


## 6. Testing

In [33]:
file_paths = {
    "semantic": "/content/drive/MyDrive/Colab Notebooks/NLP/word-testsemantic.v1.txt",
    "syntatic": "/content/drive/MyDrive/Colab Notebooks/NLP/word-testsyntatic.v1.txt"
}

def load_tests(file_path):
    with open(file_path, "r") as file:
        return [sent.strip() for sent in file]

# Load tests from files
semantic = load_tests(file_paths["semantic"])
syntatic = load_tests(file_paths["syntatic"])

In [34]:
vector_space = []

for word in vocab:
    vector_space.append(model.get_embed(word))

vector_space = np.array(vector_space)

In [35]:
#scipy version
from scipy import spatial

def cos_sim(a, b):
    cos_sim = 1 - spatial.distance.cosine(a, b)  #distance = 1 - similarlity, because scipy only gives distance
    return cos_sim

def cos_sim_scores(vector_space, target_vector):
    scores = []
    for each_vect in vector_space:
        each_vect = tuple(each_vect)
        target_vector=tuple(target_vector)
        scores.append(cos_sim(target_vector, each_vect))

    return np.array(scores)

In [36]:
def similarity(model, test_data):
    words = test_data.split(" ")

    embed0 = np.array(model.get_embed(words[0]))
    embed1 = np.array(model.get_embed(words[1]))
    embed2 = np.array(model.get_embed(words[2]))

    similar_vector = embed1 - embed0 + embed2

    similarity_scores = cos_sim_scores(vector_space, similar_vector)
    max_score_idx = np.argmax(similarity_scores)
    similar_word = index2word[max_score_idx]

    result = False
    if similar_word == words[3]:
        result = True

    return result

In [37]:
sem_total = len(semantic)
sem_correct = 0
for sent in semantic:
    if similarity(model, sent):
        sem_correct += 1

In [38]:
sem_accuracy = sem_correct / sem_total
print(f"Semantic accuracy: {sem_accuracy:2.2f}")

Semantic accuracy: 0.00


In [39]:
syn_total = len(syntatic)
syn_correct = 0
for sent in syntatic:
    if similarity(model, sent):
        syn_correct += 1

In [40]:
syn_accuracy = syn_correct / syn_total
print(f"Syntatic accuracy: {syn_accuracy:2.2f}")

Syntatic accuracy: 0.00


In [41]:
file_path = "/content/drive/MyDrive/Colab Notebooks/NLP/wordsim_similarity_goldstandard.txt"

with open(file_path, 'r') as file:
            content = file.readlines()

sim_data = []

for sent in content:
    sim_data.append(sent.strip())

In [42]:
def compute_similarity(model, test_data):
    words = test_data.split("\t")

    embed0 = np.array(model.get_embed(words[0].strip()))
    embed1 = np.array(model.get_embed(words[1].strip()))

    similarity_model = embed1 @ embed0.T
    similarity_provided = float(words[2].strip())

    return similarity_provided, similarity_model

In [43]:
ds_scores = []
model_scores = []
for sent in sim_data:
    ds_score, model_score = compute_similarity(model, sent)

    ds_scores.append(ds_score)
    model_scores.append(model_score)

In [44]:
from scipy.stats import spearmanr

corr = spearmanr(ds_scores, model_scores)[0]

print(f"Correlation between the dataset metrics and model scores is {corr:2.2f}.")

Correlation between the dataset metrics and model scores is 0.06.


## 7. Save model


In [45]:
import torch
import pickle


model_save_path = '/content/drive/MyDrive/Colab Notebooks/NLP/code/glovefromscratch.model'
torch.save(model.state_dict(), model_save_path)

model_args = {
    'voc_size': voc_size,
    'emb_size': emb_size,
    'word2index': word2index,
}

arg_file_path = '/content/drive/MyDrive/Colab Notebooks/NLP/code/glovefromscratch.args'
with open(arg_file_path, 'wb') as f:
    pickle.dump(model_args, f)

In [46]:

with open(arg_file_path, 'rb') as f:
    loaded_args = pickle.load(f)


model_glove = Glove(**loaded_args)
model_glove.load_state_dict(torch.load(model_save_path))


<ipython-input-46-7d960ee0fb84>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_glove.load_state_dict(torch.load(model_save_path))


<All keys matched successfully>

In [53]:
embedding = model_glove.get_embed('country')
print("Embedding for 'country':", embedding)


Embedding for 'country': (0.15560153126716614, -0.6647694110870361)
